In [ ]:
!nvidia-smi

In [ ]:
!unzip -q /content/dataset.zip -d /content/custom_data

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [5]:
import os
import random
import shutil
import yaml
from ultralytics import YOLO
from google.colab import drive

# ==============================================================================
# PHẦN 1: CẤU HÌNH
# ==============================================================================
drive.mount('/content/drive')

SOURCE_DATASET_DIR = "/content/custom_data/dataset"  # Toàn bộ 1936 ảnh
SPLIT_DATASET_DIR = "/content/dataset_split"
VALIDATION_SPLIT = 0.2
CLASS_NAMES = ["lisence_plate"]
EPOCHS = 100
IMAGE_SIZE = 640
BATCH_SIZE = 16  # Có thể tăng lên 32 nếu VRAM cho phép
YOLO_MODEL = 'yolov8m.pt'

# ==============================================================================
# PHẦN 2: CÁC HÀM TIỆN ÍCH
# ==============================================================================
def split_data(source_dir, output_dir, val_split):
    print(">>> BƯỚC 1: CHIA DỮ LIỆU <<<")
    source_images_dir = os.path.join(source_dir, "images")
    source_labels_dir = os.path.join(source_dir, "labels")

    if not os.path.isdir(source_images_dir) or not os.path.isdir(source_labels_dir):
        print(f"[LỖI] Không tìm thấy thư mục 'images' hoặc 'labels' trong '{source_dir}'.")
        return False

    train_images_dir = os.path.join(output_dir, "train", "images")
    train_labels_dir = os.path.join(output_dir, "train", "labels")
    val_images_dir = os.path.join(output_dir, "val", "images")
    val_labels_dir = os.path.join(output_dir, "val", "labels")

    for dir_path in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
        os.makedirs(dir_path, exist_ok=True)

    image_files = [f for f in os.listdir(source_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    random.shuffle(image_files)

    split_index = int(len(image_files) * (1 - val_split))
    train_files = image_files[:split_index]
    val_files = image_files[split_index:]

    def copy_files(files, dest_img_dir, dest_lbl_dir):
        count = 0
        for img_file in files:
            base_name = os.path.splitext(img_file)[0]
            source_img_path = os.path.join(source_images_dir, img_file)
            dest_img_path = os.path.join(dest_img_dir, img_file)
            shutil.copy(source_img_path, dest_img_path)
            count += 1

            lbl_file = f"{base_name}.txt"
            source_lbl_path = os.path.join(source_labels_dir, lbl_file)
            dest_lbl_path = os.path.join(dest_lbl_dir, lbl_file)
            if os.path.exists(source_lbl_path):
                shutil.copy(source_lbl_path, dest_lbl_path)
            else:
                with open(dest_lbl_path, 'w') as f:
                    pass
        return count

    print(f"[INFO] Sao chép {len(train_files)} file vào tập train...")
    num_train = copy_files(train_files, train_images_dir, train_labels_dir)
    print(f"[INFO] Sao chép {len(val_files)} file vào tập validation...")
    num_val = copy_files(val_files, val_images_dir, val_labels_dir)

    print("\n[THÀNH CÔNG] Chia dữ liệu hoàn tất!")
    print(f"  - Tổng số ảnh: {num_train + num_val}")
    print(f"  - Train: {num_train} ảnh")
    print(f"  - Validation: {num_val} ảnh")
    return num_train + num_val > 0

def create_yaml_file(output_dir, class_names):
    print("\n>>> BƯỚC 2: TẠO DATA.YAML <<<")
    abs_output_dir = os.path.abspath(output_dir).replace('\\', '/')
    data_config = {
        'path': abs_output_dir,
        'train': 'train/images',
        'val': 'val/images',
        'nc': len(class_names),
        'names': class_names,
        'augment': {
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'rotate': 15,
            'translate': 0.1,
            'scale': 0.7,
            'shear': 2.0,
            'mosaic': 1.0,
            'mixup': 0.5
        }
    }
    yaml_path = os.path.join(output_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f, default_flow_style=False, sort_keys=False)
    print(f"[THÀNH CÔNG] Created data.yaml at: {yaml_path}")
    return yaml_path

def evaluate_model(model, yaml_path, output_dir):
    print("\n>>> BƯỚC 4: ĐÁNH GIÁ MÔ HÌNH <<<")
    results = model.val(data=yaml_path)
    metrics = {
        'mAP@0.5': results.box.map50,
        'mAP@0.5:0.95': results.box.map,
        'Precision': results.box.p[0] if results.box.p else 0,
        'Recall': results.box.r[0] if results.box.r else 0,
        'F1-Score': results.box.f1[0] if results.box.f1 else 0
    }
    results_path = os.path.join(output_dir, 'evaluation_results.txt')
    with open(results_path, 'w') as f:
        f.write("=== Model Evaluation Results ===\n")
        for metric, value in metrics.items():
            f.write(f"{metric}: {value:.4f}\n")
    print(f"[THÀNH CÔNG] Evaluation results saved to: {results_path}")
    print("Confusion matrix và plots lưu trong 'runs/detect/yolo_bien_so_xe_detector'")
    return metrics

# ==============================================================================
# PHẦN 3: THỰC THI
# ==============================================================================
if __name__ == "__main__":
    if not split_data(SOURCE_DATASET_DIR, SPLIT_DATASET_DIR, VALIDATION_SPLIT):
        print("[LỖI] Chia dữ liệu thất bại.")
        exit()

    yaml_file_path = create_yaml_file(SPLIT_DATASET_DIR, CLASS_NAMES)

    print("\n>>> BƯỚC 3: HUẤN LUYỆN YOLOv8 <<<")
    print("="*50)
    model = YOLO(YOLO_MODEL)
    results = model.train(
        data=yaml_file_path,
        epochs=EPOCHS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        name='yolo_bien_so_xe_detector',
        device=0,
        save_period=5,
        workers=4,
        project='/content/runs',
        exist_ok=True,
        patience=30,
        lr0=0.01,
        weight_decay=0.0005
    )

    metrics = evaluate_model(model, yaml_file_path, SPLIT_DATASET_DIR)

    print("\n" + "="*50)
    print(">>> HUẤN LUYỆN HOÀN TẤT! <<<")
    print("Kết quả lưu trong '/content/runs/detect/yolo_bien_so_xe_detector'")
    print("Mô hình tốt nhất: '/content/runs/detect/yolo_bien_so_xe_detector/weights/best.pt'")
    print("Metrics:")
    for metric, value in metrics.items():
        print(f"  - {metric}: {value:.4f}")
    print("="*50)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive
>>> BƯỚC 1: CHIA DỮ LIỆU <<<
[INFO] Sao chép 1713 file vào tập train...
[INFO] Sao chép 429 file vào tập validation...

[THÀNH CÔNG] Chia dữ liệu hoàn tất!
  - Tổng số ảnh: 2142
  - Train: 1713 ảnh
  - Validation: 429 ảnh

>>> BƯỚC 2: TẠO DATA.YAML <<<
[THÀNH CÔNG] Created data.yaml at: /content/dataset_split/data.yaml

>>> BƯỚC 3: HUẤN LUYỆN YOLOv8 <<<


100%|██████████| 49.7M/49.7M [00:00<00:00, 231MB/s]


Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_bien_so_xe_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=30, perspective=0.0, plots=True, po

100%|██████████| 755k/755k [00:00<00:00, 21.0MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

...


100 epochs completed in 1.710 hours.
Optimizer stripped from /content/runs/yolo_bien_so_xe_detector/weights/last.pt, 52.0MB
Optimizer stripped from /content/runs/yolo_bien_so_xe_detector/weights/best.pt, 52.0MB

Validating /content/runs/yolo_bien_so_xe_detector/weights/best.pt...
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:05<00:00,  2.59it/s]


                   all        429        432      0.991      0.983      0.995      0.765
Speed: 0.2ms preprocess, 6.8ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to /content/runs/yolo_bien_so_xe_detector

>>> BƯỚC 4: ĐÁNH GIÁ MÔ HÌNH <<<
Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 814.9±188.1 MB/s, size: 17.8 KB)


val: Scanning /content/dataset_split/val/labels.cache... 429 images, 0 backgrounds, 0 corrupt: 100%|██████████| 429/429 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 27/27 [00:08<00:00,  3.20it/s]


                   all        429        432      0.991      0.983      0.995      0.765
Speed: 0.9ms preprocess, 14.5ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /content/runs/yolo_bien_so_xe_detector
[THÀNH CÔNG] Evaluation results saved to: /content/dataset_split/evaluation_results.txt
Confusion matrix và plots lưu trong 'runs/detect/yolo_bien_so_xe_detector'

>>> HUẤN LUYỆN HOÀN TẤT! <<<
Kết quả lưu trong '/content/runs/detect/yolo_bien_so_xe_detector'
Mô hình tốt nhất: '/content/runs/detect/yolo_bien_so_xe_detector/weights/best.pt'
Metrics:
  - mAP@0.5: 0.9947
  - mAP@0.5:0.95: 0.7655
  - Precision: 0.9907
  - Recall: 0.9832
  - F1-Score: 0.9869


In [6]:
!zip -r runs.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/yolo_bien_so_xe_detector/ (stored 0%)
  adding: content/runs/yolo_bien_so_xe_detector/train_batch1.jpg (deflated 5%)
  adding: content/runs/yolo_bien_so_xe_detector/val_batch2_labels.jpg (deflated 10%)
  adding: content/runs/yolo_bien_so_xe_detector/train_batch0.jpg (deflated 5%)
  adding: content/runs/yolo_bien_so_xe_detector/train_batch9722.jpg (deflated 13%)
  adding: content/runs/yolo_bien_so_xe_detector/F1_curve.png (deflated 19%)
  adding: content/runs/yolo_bien_so_xe_detector/weights/ (stored 0%)
  adding: content/runs/yolo_bien_so_xe_detector/weights/epoch60.pt (deflated 49%)
  adding: content/runs/yolo_bien_so_xe_detector/weights/epoch95.pt (deflated 49%)
  adding: content/runs/yolo_bien_so_xe_detector/weights/last.pt (deflated 8%)
  adding: content/runs/yolo_bien_so_xe_detector/weights/epoch55.pt (deflated 49%)
  adding: content/runs/yolo_bien_so_xe_detector/weights/epoch75.pt (deflated 49%)
  adding: content/runs/yol

In [7]:
from google.colab import files
files.download('runs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>